### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 2го знака.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [1]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [2]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return ['{:0.2f}'.format(Decimal(x) / Decimal(count)) for x in result]

#### Решение для среднего и дисперсии (общее для всех полей)

Нужно пробегаться двумя указателями и обновлять счётчики.


In [3]:
from itertools import tee
from decimal import *


getcontext().prec = 50


def stream_window_mean_and_variance(stream, window_size):
    windows_count = 0
    total_sum = 0
    total_squares_sum = 0
    total_grouped_squares_sum = 0
    
    value_sum = 0
    value_sq_sum = 0
    it_begin, it_end = tee(stream)
    
    for _ in range(window_size):
        value = next(it_end)
        value_sum += value
        value_sq_sum += value * value

    try:
        while True:
            windows_count += 1
            total_sum += value_sum
            total_squares_sum += value_sq_sum
            total_grouped_squares_sum += value_sum * value_sum

            next_value = next(it_end)
            prev_value = next(it_begin)
            
            value_sum += next_value
            value_sq_sum += next_value * next_value

            value_sum -= prev_value
            value_sq_sum -= prev_value * prev_value
    except StopIteration:
        pass
    
    mean = Decimal(total_sum) / Decimal(window_size * windows_count)
    var = Decimal(total_squares_sum) / Decimal(window_size * windows_count) - Decimal(total_grouped_squares_sum) / Decimal(window_size * window_size * windows_count)

    return '{:0.2f}    {:0.2f}'.format(mean, var)


In [4]:
%%time
stream_window_mean_and_variance(easy_stream(), 1000)

CPU times: user 1min 3s, sys: 4 ms, total: 1min 3s
Wall time: 1min 3s


'4999675.28    83439.34'

In [5]:
%%time
stream_window_mean_and_variance(medium_stream(), 1000)

CPU times: user 1min 2s, sys: 20 ms, total: 1min 2s
Wall time: 1min 3s


'127.48    5455.17'

In [6]:
%%time
stream_window_mean_and_variance(nightmare_stream(), 1000)

CPU times: user 1min 2s, sys: 8 ms, total: 1min 2s
Wall time: 1min 2s


'499880345.88    83228908564031114.59'

#### Решение для минимума, медианы и максимума для поля easy

Это поле монотонно растёт в потоке. Поэтому достаточно просто хранить deque и соответствущие индексы из него. Можно поизвращаться и сделать три указателя itertool.tee (но это надо быть совсем фанатом).


In [7]:
from collections import deque
from itertools import islice

def easy_stream_min_median_max(stream, window_size):
    queue = deque()
    for value in islice(stream, window_size):
        queue.append(value)
    
    yield queue[0], queue[window_size // 2], queue[-1]
    for value in stream:
        queue.popleft()
        queue.append(value)
        yield queue[0], queue[window_size // 2], queue[-1]

In [8]:
%%time
get_tuple_stream_mean(easy_stream_min_median_max(easy_stream(), 1000), 3)

CPU times: user 2min 41s, sys: 28 ms, total: 2min 41s
Wall time: 2min 42s


['4999175.79', '4999675.78', '5000174.76']

#### Решение для минимума, медианы и максимума для поля medium

Это поле имеет очень мало значений в потоке. Поэтому можно хранить счётчики сколько каких значений мы наблюдали.

In [9]:
from itertools import tee
from numpy import cumsum
from numpy import searchsorted

def medium_stream_min_median_max(stream, window_size):
    values_counter = np.zeros(256)
    mid_value = (window_size + 1) // 2
    it_begin, it_end = tee(stream)
    for _ in range(window_size):
        values_counter[next(it_end)] += 1
    try:
        while True:
            cum_values = cumsum(values_counter)
            yield (
                searchsorted(cum_values, 1, side='left'),
                searchsorted(cum_values, mid_value, side='right'),
                searchsorted(cum_values, window_size, side='left')
            )

            values_counter[next(it_end)] += 1
            values_counter[next(it_begin)] -= 1
    except StopIteration:
        pass

In [10]:
%%time
get_tuple_stream_mean(medium_stream_min_median_max(medium_stream(), 1000), 3)

CPU times: user 6min 23s, sys: 80 ms, total: 6min 23s
Wall time: 6min 24s


['0.02', '127.60', '254.98']

#### Решение для минимума, медианы и максимума для поля nightmare

Это поле монотонно в общем виде. По сути надо повортить решение для easy, но надо найти дополнительную библиотеку.

In [11]:
from blist import sortedlist
from itertools import islice

def nightmare_stream_min_median_max(stream, window_size):
    window = sortedlist()
    it_begin, it_end = tee(stream)
    for _ in range(window_size):
        window.add(next(it_end))
    try:
        while True:
            yield window[0], window[window_size // 2], window[-1]
            window.add(next(it_end))
            window.remove(next(it_begin))
    except StopIteration:
        pass

In [12]:
%%time
get_tuple_stream_mean(nightmare_stream_min_median_max(nightmare_stream(), 1000), 3)

CPU times: user 6min 51s, sys: 16 ms, total: 6min 51s
Wall time: 6min 52s


['1017512.29', '500438415.64', '999017359.97']

In [13]:
%%time
get_tuple_stream_mean(nightmare_stream_min_median_max(medium_stream(), 1000), 3)

CPU times: user 6min 28s, sys: 28 ms, total: 6min 28s
Wall time: 6min 28s


['0.02', '127.60', '254.98']

In [14]:
%%time
get_tuple_stream_mean(nightmare_stream_min_median_max(easy_stream(), 1000), 3)

CPU times: user 6min 17s, sys: 12 ms, total: 6min 17s
Wall time: 6min 18s


['4999175.79', '4999675.78', '5000174.76']